In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from scipy import ndimage
from func import *
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd

/home/mainak/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/mainak/.local/lib/python2.7/site-packages/pyneb/utils/manage_atomic_data.py:528: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  gsconf = np.genfromtxt(execution_path('../atomic_data/levels/gsconfs.dat'), names=['atom', 'gsconf'], dtype=None)


In [2]:
def source_tab(obj,z,d_a):
    a1 = np.array(['%s'%(obj)])
    a2 = np.array([z])
    a3 = np.array([d_a])
    hdu = fits.BinTableHDU.from_columns([fits.Column(name='Source', format='20A', array=a1),
          fits.Column(name='z',format='E',array=a2),fits.Column(name='d_a',format='E',array=a3)])
    hdu.writeto('Source_name_%s.fits'%(obj),overwrite=True) 
    
def source_name_tab(obj):
    a1 = np.array(['%s'%(obj)])
    hdu = fits.BinTableHDU.from_columns([fits.Column(name='Source', format='20A', array=a1)])
    hdu.writeto('source_%s.fits'%(obj),overwrite=True)  

def AGN_energetics_table(obj):
    t1 = Table.read('source_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_energetics_table.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_energetics.fits'%(obj),overwrite=True)

In [3]:
def algorithm_script(obj,z,d_a,prefix_path_cube="/home/mainak/xdata/ftp.hidrive.strato.com/users/login-carsftp"):
    source_tab(obj,z,d_a)
    source_name_tab(obj)
    AGN_energetics_table(obj)

In [4]:
z = {"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0119-0118":0.054341
     ,"HE0224-2834":0.059800,"HE0253-1641":0.031588,"HE0433-1028":0.035550}

d_a = {"HE0040-1105":170.9,"HE0108-4743":100.1,"HE0119-0118":218.1
       ,"HE0224-2834":238.5,"HE0253-1641":130.2,"HE0433-1028":145.9}

#z_remaining = {"HE2128-0221":0.05248,"HE1248-1356":0.01465}

objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],d_a[obj])

In [5]:
def master_table(input_filename,output_filename):
    infiles = sorted(glob.glob(input_filename)) # './*source.fits'
    infiles
    tabledict = {}

    for i, file in enumerate(infiles):
        hdulist = fits.open(file)
        bintab = hdulist[1].data
        table = Table(bintab)
        name = table['Source'][0]  
        tabledict[name] = table

    tab = vstack(list(tabledict.values()))
    df = tab.to_pandas()
    df = df.sort_values(by='Source', ascending=True)
    dtab = Table.from_pandas(df)
    dtab.write(output_filename, format='fits', overwrite=True)

In [6]:
master_table(input_filename='./*energetics.fits',output_filename='outflow_energetics_table.fits')